In [6]:
import requests 
import time
from tqdm import tqdm
import json
import pandas as pd
import os
from fake_useragent import UserAgent

In [7]:
collection = 'CEA-2d29f9'
chunk_size = 50
sleep_time = 0.4

In [8]:
data = pd.read_csv(f'{collection}_nfts.csv')

In [9]:
if f'{collection}_transactions.json' in os.listdir():
    with open(f'{collection}_transactions.json', 'r') as f:
        transactions_data = json.load(f)
else:
    transactions_data = {}

In [13]:
for _, nft in tqdm(data.iterrows(), total=data.shape[0]):
    identifier = nft['identifier']
    if identifier in transactions_data and transactions_data[identifier]['count'] == len(transactions_data[identifier]['transactions']):
        continue
    transactions_data[identifier] = {
        'count': -1,
        'transactions': []
    }
    time.sleep(sleep_time)
    url = f'https://api.elrond.com/nfts/{identifier}/transactions/count'
    user_agent = UserAgent().random
    headers = {'User-Agent': user_agent}
    count = requests.get(url, headers=headers).json()
    transactions_data[identifier]['count'] = count
    for index in range(0, count, chunk_size):
        time.sleep(sleep_time)
        url = f'https://api.elrond.com/nfts/{identifier}/transactions?from={index}&size={chunk_size}'
        user_agent = UserAgent().random
        headers = {'User-Agent': user_agent}
        transactions_data[identifier]['transactions'] += requests.get(url, headers=headers).json()

100%|██████████| 2043/2043 [00:00<00:00, 22584.31it/s]


In [14]:
with open(f'{collection}_transactions.json', 'w') as f:
    json.dump(transactions_data, f, indent=4)

In [15]:
c = 0 
for identifier in transactions_data:
    c += transactions_data[identifier]['count']
c

20762